# Vector Enrichment Jobs with Amazon SageMaker Geospatial Capabilities

---

This notebook's CI test result for us-west-2 is as follows. CI test results in other regions can be found at the end of the notebook. 

![This us-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-2/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

---

In this notebook we will take a look at the [vector enrichment](https://docs.aws.amazon.com/sagemaker/latest/dg/geospatial-vej.html) operations available as part of SageMaker's new Geospatial capabilities.

----

## Environment Set-Up

If you have not already done so, please make sure you have opened this notebook in [**SageMaker Studio**](https://us-west-2.console.aws.amazon.com/sagemaker/home?region=us-west-2#/studio) in the **Oregon (us-west-2)** region. The Geospatial capabilities of SageMaker are currently only available in this region.

Please also ensure that you have set:
- the notebook image to **Geospatial 1.0**
- the kernel to **Python 3**
- the instance type to **ml.m5.4xlarge**. 

We will start by making sure the "sagemaker" SDK is updated, and then import the libraries we will need for the notebook.

In [ ]:
!pip install sagemaker --upgrade

In [ ]:
import boto3
import sagemaker
import folium
import json
from datetime import datetime, timedelta
from dateutil import parser
import time
import os
import random

Next let's ensure that our working directory is the same as the 'files' folder in our Studio Notebook environment.

In [ ]:
os.chdir("/root/data")

We will now define a few variables that we will use throughout the notebook that relate to AWS resources.

In [ ]:
sagemaker_session = sagemaker.Session()
bucket_name = (
    sagemaker_session.default_bucket()
)  ### Replace with your own bucket if needed
role = sagemaker.get_execution_role(sagemaker_session)
s3prefix = "sm-geospatial-vej"  ### Replace with the S3 prefix desired
region = boto3.Session().region_name
print(f"S3 bucket: {bucket_name}")
print(f"Role: {role}")
print(f"Region: {region}")

**Note, at the time of writing this notebook sagemaker-geospatial is only available in the region "us-west-2".**

Make sure you have the proper policy attached to your notebook's execution role, and a trust relationship added to your role for "sagemaker-geospatial", as specified in the [Get Started with Amazon SageMaker Geospatial Capabilities documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/geospatial-getting-started.html).

In [ ]:
gsClient = boto3.client("sagemaker-geospatial", region_name=region)

In [ ]:
s3Client = boto3.Session().resource(service_name="s3")

----

## Map Matching

**Map Matching** allows you to snap GPS coordinates to road segments. The input should be a CSV file containing the trace ID (route), longitude, latitude and the timestamp attributes. Let's put a CSV in our working directory to send to S3 later.

In [ ]:
mm_input_csv = """id,longitude,latitude,timestamp
0,-0.158905816,51.51341681,2023-01-04T17:46:33.127473
0,-0.165058309,51.51272588,2023-01-04T17:46:53.127473
0,-0.168111426,51.51215009,2023-01-04T17:47:13.127473
0,-0.169406688,51.51197736,2023-01-04T17:47:33.127473
0,-0.172552324,51.51122882,2023-01-04T17:47:53.127473
0,-0.172135989,51.51065302,2023-01-04T17:48:13.127473
0,-0.170563172,51.50883919,2023-01-04T17:48:33.127473
0,-0.170748209,51.5069677,2023-01-04T17:48:53.127473
0,-0.173061176,51.5057008,2023-01-04T17:49:13.127473
0,-0.174633994,51.50282136,2023-01-04T17:49:33.127473
0,-0.173061176,51.50175592,2023-01-04T17:49:53.127473
0,-0.170424393,51.5019287,2023-01-04T17:50:13.127473
0,-0.165104568,51.50171351,2023-01-04T17:50:33.127473
0,-0.161614045,51.5018711,2023-01-04T17:50:53.127473
0,-0.160293596,51.50210147,2023-01-04T17:51:13.127473
0,-0.158720778,51.50233184,2023-01-04T17:51:33.127473
0,-0.154783224,51.50280618,2023-01-04T17:51:53.127473
0,-0.1493934562836614,51.502530034103216,2023-01-04T17:52:13.127473
0,-0.1455683526539815,51.50242490130108,2023-01-04T17:52:33.127473
0,-0.14159641737705897,51.502158768076725,2023-01-04T17:52:53.127473
0,-0.14024485606748271,51.501987068397625,2023-01-04T17:53:13.127473
"""

In [ ]:
import os

os.makedirs("./vej-input", exist_ok=True)
open("./vej-input/mm-input.csv", "w").write(mm_input_csv)

print(mm_input_csv)

As you can see, there are four columns. The first is the "id" column, which holds the "trace"/"drive" ID of the route that is being map-matched. Here the only value is 0 because the data only pertains to one single drive. It is possible to supply data that has many routes, in which case the map matching for each route is calculated separately.

The next two columns hold the longitude and latitude of the GPS system at a that particular moment of the route. Finally, we have the timestamp column, which indicates the time at which that particular coordinate was reached on the route, and allows SageMaker to determine the order in which coordinates appear on the route.

### Visualizing input data on a road map

Let's take a look at the data on a map. We will use the `folium` Python library to do this.

First we will need to parse the CSV to extract the longitude and latitude coordinates.

In [ ]:
def csv_parse(csv_string):
    rows = csv_string.split("\n")
    column_names = rows[0].split(",")
    parsed_rows = []

    for r, row in enumerate(rows):
        if r == 0:
            continue

        columns = row.split(",")

        if len(columns) <= 1:
            continue

        parsed_row = dict()

        for c, column_name in enumerate(column_names):
            parsed_row[column_name.strip()] = columns[c].strip()

        parsed_rows.append(parsed_row)

    return parsed_rows

In [ ]:
mm_input = csv_parse(mm_input_csv)
mm_input

`folium` expects coordinates as a list of `[latitude, longitude]`.

In [ ]:
mm_input_folium = []
mm_input_coords = []

for mm_input_row in mm_input:
    mm_input_folium.append(
        [float(mm_input_row["latitude"]), float(mm_input_row["longitude"])]
    )
    mm_input_coords.append(
        [float(mm_input_row["longitude"]), float(mm_input_row["latitude"])]
    )

mm_input_coords

Finally, we will need to provide `folium` with the area of interest to zoom to. Let's supply the bounding box of the coordinates we have extracted from the input file.

In [ ]:
minLat, minLon = 1000, 1000
maxLat, maxLon = -1000, -1000

for coord in mm_input_folium:
    if coord[0] > maxLat:
        maxLat = coord[0]
    if coord[0] < minLat:
        minLat = coord[0]
    if coord[1] > maxLon:
        maxLon = coord[1]
    if coord[1] < minLon:
        minLon = coord[1]

sw = [minLat, minLon]
ne = [maxLat, maxLon]

print(sw)
print(ne)

With all that done, let's render the coordinates on the map.

In [ ]:
def init_map():
    vej_map = folium.Map(
        zoom_control=False, no_touch=True, scrollWheelZoom=False, dragging=False
    )
    vej_map.fit_bounds(bounds=[sw, ne])
    folium.TileLayer("cartodbpositron").add_to(vej_map)
    for coord in mm_input_folium:
        folium.Circle(location=coord, radius=10, color="red").add_to(vej_map)

    return vej_map


vej_map = init_map()
vej_map

The points rendered on the map show a coordinate trace through the City of London. Note how some points aren't perfectly sat on the roads that the trace likely follows.

### Starting the Vector Enrichment Job

SageMaker expects this data to be available on S3. As such, let's upload that file to our S3 bucket.

In [ ]:
s3Client.Bucket(bucket_name).Object(f"{s3prefix}/{mm_input_filename}").upload_file(
    mm_input_filename
)

Let's now define some methods that will allow us to easily call the Vector Enrichment Job API.

In [ ]:
def start_vector_enrichment_job(vej_name, vej_input_config, vej_config):
    # Start VEJ...
    response = gsClient.start_vector_enrichment_job(
        Name=vej_name,
        ExecutionRoleArn=role,
        InputConfig=vej_input_config,
        JobConfig=vej_config,
    )
    vej_arn = response["Arn"]
    print(f"{datetime.now()} - Started VEJ: {vej_arn}")

    # Wait for VEJ to complete... check status every minute
    gs_get_vej_resp = {"Status": "IN_PROGRESS"}
    while gs_get_vej_resp["Status"] == "IN_PROGRESS":
        time.sleep(30)
        gs_get_vej_resp = gsClient.get_vector_enrichment_job(Arn=vej_arn)
        print(f'{datetime.now()} - Current VEJ status: {gs_get_vej_resp["Status"]}')
    return vej_arn

In [ ]:
def create_input_config(fname):
    return {
        "DocumentType": "CSV",
        "DataSourceConfig": {
            "S3Data": {"S3Uri": f"s3://{bucket_name}/{s3prefix}/{fname}"}
        },
    }

With our helper methods defined, let's initialize the config objects required by the map matching operation.

In [ ]:
map_matching_input_config = create_input_config(mm_input_filename)
map_matching_input_config

In [ ]:
map_matching_job_config = {
    "MapMatchingConfig": {
        "IdAttributeName": "id",
        "TimestampAttributeName": "timestamp",
        "XAttributeName": "longitude",
        "YAttributeName": "latitude",
    }
}
map_matching_job_config

We can now start the Map Matching operation. This may take a few minutes, but thankfully the function we defined to start the job will print out the status of the job every 30 seconds.

In [ ]:
vej_arn = start_vector_enrichment_job(
    f"mapmatch_{datetime.now().isoformat()}",
    map_matching_input_config,
    map_matching_job_config,
)
vej_arn

### Exporting the Vector Enrichment Job

With the job completed, we would now like to view the results. The results are currently stored within SageMaker, and require an export to S3 for us to easily access them. Let's define a function that will start the export, and similarly print out the export status as it updates.

In [ ]:
def export_vector_enrichment_job(arn, outfoldername):
    # Start VEJ export...
    response = gsClient.export_vector_enrichment_job(
        Arn=arn,
        ExecutionRoleArn=role,
        OutputConfig={
            "S3Data": {"S3Uri": f"s3://{bucket_name}/{s3prefix}/{outfoldername}"}
        },
    )

    print(response)

    vej_arn = response["Arn"]
    print(f"{datetime.now()} - Started VEJ export: {vej_arn}")

    # Wait for VEJ export to complete... check status every minute
    gs_get_vej_resp = {"ExportStatus": "IN_PROGRESS"}
    while gs_get_vej_resp["ExportStatus"] == "IN_PROGRESS":
        time.sleep(30)
        gs_get_vej_resp = gsClient.get_vector_enrichment_job(Arn=vej_arn)
        print(
            f'{datetime.now()} - Current VEJ export status: {gs_get_vej_resp["ExportStatus"]}'
        )

    return response["OutputConfig"]["S3Data"]["S3Uri"]

In [ ]:
mm_outfolder = "vej-output/mm-output/"

In [ ]:
export_s3_uri = export_vector_enrichment_job(vej_arn, mm_outfolder)
export_s3_uri

Let's download the results to our Notebook directory so that we can look at them.

In [ ]:
def download_s3_directory_latest(directory):
    # The export job adds some random characters to the key
    #   e.g. 'vej-output/mapmatching-output/3rmeymfupxf9/links.geojson'
    #   this is so that exports don't overwrite each other.
    # We don't have a way to directly obtain the subdirectory of the latest export
    #   so let's get the last modified versions in the output directory
    latest_dict = dict()
    bucket = s3Client.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=f"{s3prefix}/{directory}"):
        fname = obj.key.split("/")[-1]
        if latest_dict.get(fname) == None:
            latest_dict[fname] = {"last_modified": obj.last_modified, "key": obj.key}
        else:
            this_last_modified = (
                parser.parse(obj.last_modified)
                if isinstance(obj.last_modified, str)
                else obj.last_modified
            )

            dict_last_modified = latest_dict[fname]["last_modified"]
            if this_last_modified > dict_last_modified:
                latest_dict[fname] = {
                    "last_modified": this_last_modified,
                    "key": obj.key,
                }

    # the export job adds some random characters to the key
    # e.g. 'vej-output/mapmatching-output/3rmeymfupxf9/links.geojson'
    # we will remove them here: => 'vej-output/mapmatching-output/links.geojson'
    for fname in latest_dict:
        print(latest_dict[fname]["key"])
        print(latest_dict[fname]["last_modified"])

        obj_key = latest_dict[fname]["key"]
        key_parts = obj_key.split("/")
        local_name = f"{key_parts[1]}/{key_parts[2]}/{key_parts[4]}"
        if not os.path.exists(os.path.dirname(local_name)):
            os.makedirs(os.path.dirname(local_name))
        bucket.download_file(obj_key, local_name)

        print(local_name)
        print()

In [ ]:
download_s3_directory_latest(mm_outfolder)

### Visualizing map match output on a road map

With the map matching completed, let's view the snapped coordinates on the map in comparison to our input coordinates.

The snapped coordinates live in the `waypoints.geojson` file. `geojson` is a commonly used data format for Geospatial data.

In [ ]:
f = open(f"{mm_outfolder}waypoints.geojson", "r")
waypoints = json.loads(f.read())
f.close()

waypoints

In order to plot the waypoint on our `folium` map, we need to convert them to a list of [latitude, longitude], (here they are in [longitude, latitude]).

In [ ]:
waypoint_features = waypoints["features"]
waypoint_coords_folium = []
for feature in waypoint_features:
    waypoint_coords_folium.append(
        [feature["geometry"]["coordinates"][1], feature["geometry"]["coordinates"][0]]
    )

waypoint_coords = []
for feature in waypoint_features:
    waypoint_coords.append(
        [feature["geometry"]["coordinates"][0], feature["geometry"]["coordinates"][1]]
    )

waypoint_coords

Since we want to compare the snapping results to out input, let's plot the snapped line on our existing map.

In [ ]:
def draw_waypoints(m):
    for coord in waypoint_coords_folium:
        folium.Circle(location=coord, radius=10, color="blue").add_to(m)
    return m


vej_map = draw_waypoints(init_map())
vej_map

As before, the red points are our input coordinates, and the new blue ones are the snapped coordinates from the map matching job output. As you can see, the points that were not quite on the adjacent roads have now been snapped perfectly onto the roads.

Okay, so now we have seen the road-snapped coordinates that we found in the `waypoints.geojson` output file, but what about the `links.geojson` output file? What does that contain?

The links file contains coordinate information about all the known roads segments along the shortest viable path that allows us to reach all of our waypoints in one drive (if possible).

First let's open the links file, parse it to JSON, and look at the contents.

In [ ]:
f = open(f"{mm_outfolder}links.geojson", "r")
links = json.loads(f.read())
f.close()

links

As you can see it is a pretty long file. What we care about is the stuff in the 'features' member, a list of line features, each describing a road segment. Let's extract the coordinates of each line segment into a list of lists of coordinates. Remember again that `folium` expects the coordinates as `[latitude, longitude]` so we need to swap the coordinates around.

In [ ]:
link_features = links["features"]
link_lines = []


def swap(coord_pair):
    return [coord_pair[1], coord_pair[0]]


for feature in link_features:
    coords = feature["geometry"]["coordinates"]
    coords = list(map(swap, coords))
    link_lines.append(coords)

link_lines

We can see that there are many links on the route. Each link is an individual road segment. A road is divided into separate segments at points where it connects to other roads.

For example, a long straight road which has a junction in the middle could be divided into three segments: the part before the junction, the junction, and the part after the junction. This is why long roads that have many connections to side streets contain many individual road segments/links.

In [ ]:
color1 = (255, 165, 0)
color2 = (238, 130, 238)


def color_grad(i):
    nLinks = len(link_lines)
    weight = (i + 0.5) / nLinks

    def av(x1, x2):
        return x1 * (1 - weight) + x2 * weight

    color = (
        av(color1[0], color2[0]),
        av(color1[1], color2[1]),
        av(color1[2], color2[2]),
    )

    return f"rgb({color[0]},{color[1]},{color[2]})"

In [ ]:
def draw_links(m):
    for i, link_line in enumerate(link_lines):
        folium_link_line = folium.PolyLine(
            locations=link_line, weight=5, color=color_grad(i)
        )
        folium_link_line.add_to(m)

    for coord in waypoint_coords_folium:
        folium.Circle(location=coord, radius=10, color="blue").add_to(m)

    return m


vej_map = draw_links(draw_waypoints(init_map()))
vej_map

Here we have rendered the link data onto the map such that the start of the calculated route is in orange and the end is in pink.

Notice how the path at the start of the route differs from what we might expect when looking at the waypoint data. One might think the westward drive along the north of Hyde Park would be an unbroken straight line. Instead, the route is altered such that we go around the oval at the north-east of Hyde Park in order to get back on Bayswater Road. 

This is because the link output takes road directionality into consideration. The connection between eastern Bayswater Road and western Oxford Street is one-directional, allowing only eastward traffic. As such, the shortest viable route that allows the waypoints to be joined requires we take the loop adjacent to north-east Hyde Park.

Let's render the two maps side-by-side so that you can compare.

## Reverse Geocoding

**Reverse Geocoding** is the process of deriving human-readable addresses from coordinates. Let's take a look at how to do this in SageMaker.

The Reverse Geocoding operation in SageMaker requires a CSV of coordinates as input. Let's use the snapped coordinates that we computed in the map match.

In [ ]:
rg_input_filename = "vej-input/revgeo-input.csv"
f = open(f"./{rg_input_filename}", "w")
csv = "longitude, latitude\n"

for coord in waypoint_coords_folium:
    csv += f"{coord[1]}, {coord[0]}\n"

f.write(csv)
f.close()

Again, we need to upload our CSV to S3 in order for it to be accessible by SageMaker.

In [ ]:
s3Client.Bucket(bucket_name).Object(f"{s3prefix}/{rg_input_filename}").upload_file(
    rg_input_filename
)

Like the map matching job, the reverse geocoding job will require an input file config and a job config to be passed to the start Vector Enrichment Job function.

In [ ]:
rg_input_config = create_input_config(rg_input_filename)
rg_input_config

In [ ]:
rg_job_config = {
    "ReverseGeocodingConfig": {
        "XAttributeName": "longitude",
        "YAttributeName": "latitude",
    }
}

As with the map matching job, we then pass the config objects to our start Vector Enrichment Job function, and export the results to s3 once it is completed.

In [ ]:
vej_arn = start_vector_enrichment_job(
    f"revgeo_{datetime.now().isoformat()}", rg_input_config, rg_job_config
)
vej_arn

In [ ]:
rg_outfolder = "vej-output/revgeo-output/"
response = export_vector_enrichment_job(vej_arn, rg_outfolder)
response

Now that the results are exported to S3, let's download them to our notebook's working directory so that we can visualize the results on the map.

In [ ]:
download_s3_directory_latest(rg_outfolder)

In [ ]:
f = open(f"./{rg_outfolder}/results_0.csv", "r")
rg_results_csv = f.read()
print(rg_results_csv)
f.close()

The result is a CSV whose first two columns are identical to what we supplied as input, and with extra columns added by the reverse geocoding job that contain human-readable address properties.

Let's parse this CSV so that we can easier work with it.

In [ ]:
rg_results = csv_parse(rg_results_csv)
rg_results

Now we can show the results on a map. To avoid the map being too saturated with markers, we will only plot every third address.

In [ ]:
from folium.features import DivIcon


def draw_markers(m):
    def addressHtml(result):
        html = """
            <div style="
                font-size: 7pt; 
                color: black;
                background: rgba(255,255,255,0.6); 
                border-radius: 0px 12px 42px 12px; 
                padding: 4px;
                border: 1px solid rgba(80,80,80,1.0);
                box-shadow: 2px 2px 10px rgba(0, 0, 0, 0.3);
                display: inline-block;
            ">
                <div>reverse_geo.label</div>
                <div>reverse_geo.municipality</div>
                <div>reverse_geo.neighborhood</div>
                <div>reverse_geo.region</div>
            </div>
        """

        for key in result:
            html = html.replace(key, result[key].strip('"'))

        return html

    for i in range(0, len(rg_results) // 3):
        result = rg_results[i * 3]
        label = result["reverse_geo.label"]
        road = result["reverse_geo.municipality"]
        city = result["reverse_geo.neighborhood"]
        postcode = result["reverse_geo.region"]
        folium.Marker(
            location=[result["latitude"], result["longitude"]],
            icon=DivIcon(
                icon_size=(100, 100),
                icon_anchor=(0, 0),
                html=addressHtml(result),
            ),
        ).add_to(m)

    return m


vej_map = draw_markers(draw_links(draw_waypoints(init_map())))
vej_map

That's it! Thank you for following this notebook on Vector Enrichment Jobs in SageMaker!

----

## Cleanup

If you want to remove the objects saved in your S3 bucket as part of this notebook, uncomment and run the following code.

In [ ]:
#!aws s3 rm s3://{bucket}/{prefix} --recursive

## Notebook CI Test Results

This notebook was tested in multiple regions. The test results are as follows, except for us-west-2 which is shown at the top of the notebook.

![This us-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This us-east-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-east-2/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This us-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/us-west-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This ca-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ca-central-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This sa-east-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/sa-east-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This eu-west-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This eu-west-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-2/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This eu-west-3 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-west-3/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This eu-central-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-central-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This eu-north-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/eu-north-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This ap-southeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This ap-southeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-southeast-2/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This ap-northeast-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This ap-northeast-2 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-northeast-2/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)

![This ap-south-1 badge failed to load. Check your device's internet connectivity, otherwise the service is currently unavailable](https://prod.us-west-2.tcx-beacon.docs.aws.dev/sagemaker-nb/ap-south-1/sagemaker-geospatial|london-mapmatch-and-reverse-geocode|london-mapmatch-and-reverse-geocode.ipynb)
